In [1]:
import sqlite3 as sql
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

In [2]:
df = pd.read_csv('data/motor_data11-14lats.csv', sep = ',' )

In [3]:
df

,SEX,INSR_BEGIN,INSR_END,EFFECTIVE_YR,INSR_TYPE,INSURED_VALUE,PREMIUM,OBJECT_ID,PROD_YEAR,SEATS_NUM,CARRYING_CAPACITY,TYPE_VEHICLE,CCM_TON,MAKE,USAGE,CLAIM_PAID
0,0,08-AUG-13,07-AUG-14,08,1202,519755.22,7209.140,5000029885,2007.0,4.0,6.0,Pick-up,3153.0,NISSAN,Own Goods,NaN
1,0,08-AUG-12,07-AUG-13,08,1202,519755.22,7203.890,5000029885,2007.0,4.0,6.0,Pick-up,3153.0,NISSAN,Own Goods,NaN
2,0,08-AUG-11,07-AUG-12,08,1202,519755.22,7045.804,5000029885,2007.0,4.0,6.0,Pick-up,3153.0,NISSAN,Own Goods,NaN
3,0,08-JUL-11,07-AUG-11,08,1202,519755.22,287.250,5000029885,2007.0,4.0,6.0,Pick-up,3153.0,NISSAN,Own Goods,NaN
4,0,08-AUG-13,07-AUG-14,08,1202,285451.24,4286.900,5000029901,2010.0,4.0,7.0,Pick-up,2494.0,TOYOTA,Own Goods,19894.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293532,2,11-AUG-12,10-AUG-13,88,1201,0.00,577.700,5000047311,1985.0,4.0,NaN,Automobile,1295.0,TOYOTA,Private,NaN
293533,2,11-AUG-11,10-AUG-12,88,1201,0.00,695.318,5000047311,1985.0,4.0,NaN,Automobile,1295.0,TOYOTA,Private,NaN
293534,2,01-JUN-13,31-MAY-14,13,1201,250000.00,4778.450,5000949222,1993.0,4.0,NaN,Automobile,1295.0,TOYOTA,Private,NaN
293535,0,08-JUL-12,07-JUL-13,85,1202,0.00,1057.427,5000049231,2017.0,2.0,0.0,Truck,4570.0,ISUZU,General Cartage,NaN


In [4]:
#Crear columna Id
df['Id'] = range(1, len(df) + 1)

In [5]:
con = sql.connect("data/Base_db.sqlite") #esto crea una nueva base dentro de data y si ya existe solo la llama.
# cuando creo una coneccion estoy creando un objeto con metodos.

#activa el metodo cursor que permite ejecutar codigo de sql
cursor = con.cursor()

In [6]:
# subir el DataFrame en SQL
df.to_sql('Seguros_de_autos', con, if_exists='replace', index=False)

293537

In [7]:
#consultar las tablas que tenemos
cursor.execute(
    '''
    SELECT tbl_name
    FROM sqlite_master
    WHERE type = 'table'
    '''
)

#cursor.fetchall() lo que hace es consultar las tablas que tenemos
print(cursor.fetchall())

[('Seguros_de_autos',)]


In [8]:
# Crear datos nulos en la columna PREMIUM para filas con un 9 en su Id
cursor.execute("""
--Seleccionar la tabla Seguros_de_autos para modifica
UPDATE Seguros_de_autos
SET PREMIUM = CASE
    WHEN Id LIKE '%9%' THEN NULL
    ELSE PREMIUM
END

""")
con.commit()

In [9]:
#Agregar columna rasgo_asegurado en la tabla Seguros_de_autos
cursor.execute("""
ALTER TABLE Seguros_de_autos
ADD COLUMN rango_asegurado VARCHAR(50);

""")

In [10]:
# Discretice la variable INSURED_VALUE
cursor.execute("""
--Seleccionar la tabla Seguros_de_autos para modifica
UPDATE Seguros_de_autos
SET rango_asegurado = CASE
    WHEN INSURED_VALUE < 1000 THEN '1e3'
    WHEN INSURED_VALUE >= 1000 AND INSURED_VALUE < 1000000 THEN '1e6'
    WHEN INSURED_VALUE >= 1000000 AND INSURED_VALUE < 1000000000 THEN '1e9'
    ELSE '1e12'
END;

""")
con.commit()

# Agrupación Grupo 1

### Promedio

In [11]:
#Crea tabla temporal del los posibles grupos de las combinaciones de las columas con el promedio
cursor.execute("""
-- Tabla temporal para guardar el promedio de la agrupación
CREATE TEMPORARY TABLE agru_prom AS
SELECT
    SEX,
    PROD_YEAR,
    SEATS_NUM,
    rango_asegurado,
    AVG(PREMIUM) AS prom_PREMIUM 
    -- Guarda los valores en esta columna de los promedio por grupo
    -- Es el promedio para cada grupo de combi de columnas
FROM
    Seguros_de_autos
WHERE
    PREMIUM IS NOT NULL
    
--Agrupa las filas de la tabla según las columnas especificadas   
--Es decir, va a calcular el promedio PREMIUM dentro de cada grupo
GROUP BY           
    SEX, PROD_YEAR, SEATS_NUM, rango_asegurado;
""")
con.commit()

In [12]:
#Crear una tabla temporal copia de seguros de autos para guardar la tabla con los datos imputados con con la agrupación y estadística especificada
cursor.execute("""
CREATE TEMPORARY TABLE tempora_gru1_prom AS
SELECT *
FROM Seguros_de_autos;
""")

In [13]:
# (1) Filtrar filas para actualización: WHERE Seguros_de_autos.PREMIUM IS NULL
# Identificar todas las filas en la tabla Seguros_de_autos donde la columna PREMIUM es nula.
# El motor de la base de datos primero evalúa esta condición para determinar qué filas serán potencialmente actualizadas.

# (2) Subconsulta para obtener el valor: SELECT agru_prom.prom_PREMIUM ...
# Para cada fila identificada en el paso 1, ejecutar esta subconsulta para encontrar el valor prom_PREMIUM correspondiente en la tabla agru_prom.
# Esta subconsulta se ejecuta para cada fila que cumple la condición WHERE Seguros_de_autos.PREMIUM IS NULL.

# (3) Asignación del valor: SET Seguros_de_autos.PREMIUM = ...
# Asignar el valor obtenido de la subconsulta a la columna PREMIUM de Seguros_de_autos.
# Si la subconsulta encuentra un valor prom_PREMIUM, este se asigna a la columna PREMIUM de la fila actual en Seguros_de_autos.

# (4) Actualización de la tabla: UPDATE Seguros_de_autos
# Realizar la operación de actualización en la tabla Seguros_de_autos.
# Finalmente, el motor de la base de datos actualiza las filas de Seguros_de_autos que cumplen con la condición WHERE Seguros_de_autos.PREMIUM IS NULL con el valor obtenido de la subconsulta.


cursor.execute("""
UPDATE tempora_gru1_prom
SET PREMIUM = (
    SELECT agru_prom.prom_PREMIUM
    FROM agru_prom
    WHERE
        tempora_gru1_prom.SEX = agru_prom.SEX AND
        tempora_gru1_prom.PROD_YEAR = agru_prom.PROD_YEAR AND
        tempora_gru1_prom.SEATS_NUM = agru_prom.SEATS_NUM AND
        tempora_gru1_prom.rango_asegurado = agru_prom.rango_asegurado
)
WHERE tempora_gru1_prom.PREMIUM IS NULL;
""")
con.commit()


In [14]:
#cursor.execute("""
#DROP TABLE tabla_agrupa
#""")

### Máximo

In [15]:
#Crea tabla temporal del los posibles grupos de las combinaciones de las columas
cursor.execute("""
-- Tabla temporal para guardar el promedio de la agrupación
CREATE TEMPORARY TABLE agru_max AS
SELECT
    SEX,
    PROD_YEAR,
    SEATS_NUM,
    rango_asegurado,
    MAX(PREMIUM) AS max_PREMIUM 
    -- Guarda los valores en esta columna deL máximo por grupo
    -- Es el max para cada grupo de combi de columnas
FROM
    Seguros_de_autos
WHERE
    PREMIUM IS NOT NULL
    
--Agrupa las filas de la tabla según las columnas especificadas   
--Es decir, va a calcular el max PREMIUM dentro de cada grupo
GROUP BY           
    SEX, PROD_YEAR, SEATS_NUM, rango_asegurado;
""")
con.commit()

In [16]:
#Crear una tabla temporal copia de seguros de autos para guardar la tabla con los datos imputados con con la agrupación y estadística especificada
cursor.execute("""
CREATE TEMPORARY TABLE tempora_gru1_max AS
SELECT *
FROM Seguros_de_autos;
""")

In [17]:
#modificar los nulos con el maximo
cursor.execute("""
UPDATE tempora_gru1_max
SET PREMIUM = (
    SELECT agru_max.max_PREMIUM
    FROM agru_max
    WHERE
        tempora_gru1_max.SEX = agru_max.SEX AND
        tempora_gru1_max.PROD_YEAR = agru_max.PROD_YEAR AND
        tempora_gru1_max.SEATS_NUM = agru_max.SEATS_NUM AND
        tempora_gru1_max.rango_asegurado = agru_max.rango_asegurado
)
WHERE tempora_gru1_max.PREMIUM IS NULL;
""")
con.commit()

# Imputación Grupo 2

### Promedio

In [18]:
cursor.execute("""
-- Tabla temporal para guardar el promedio de la agrupación
CREATE TEMPORARY TABLE tabla_agrupa_prom AS
SELECT
    PROD_YEAR,
    SEATS_NUM,
    TYPE_VEHICLE,
    rango_asegurado,
    AVG(PREMIUM) AS imput_PREMIUM_prom 
FROM
    Seguros_de_autos
WHERE
    PREMIUM IS NOT NULL
    
--Agrupa las filas de la tabla según las columnas especificadas   
--Es decir, va a calcular el promedio PREMIUM dentro de cada grupo
GROUP BY           
    PROD_YEAR, SEATS_NUM, TYPE_VEHICLE, rango_asegurado;
""")
con.commit()

In [19]:
#Crear una tabla temporal copia de seguros de autos para guardar la tabla con los datos imputados con con la agrupación y estadística especificada
cursor.execute("""
CREATE TEMPORARY TABLE tempora_gru2_prom AS
SELECT *
FROM Seguros_de_autos;
""")

In [20]:
#modificar los nulos con el promedio
cursor.execute("""
UPDATE tempora_gru2_prom
SET PREMIUM = (
    SELECT tabla_agrupa_prom.imput_PREMIUM_prom
    FROM tabla_agrupa_prom
    WHERE
        tempora_gru2_prom.PROD_YEAR = tabla_agrupa_prom.PROD_YEAR AND
        tempora_gru2_prom.SEATS_NUM = tabla_agrupa_prom.SEATS_NUM AND
        tempora_gru2_prom.rango_asegurado = tabla_agrupa_prom.rango_asegurado AND
        tempora_gru2_prom.TYPE_VEHICLE = tabla_agrupa_prom.TYPE_VEHICLE
)
WHERE tempora_gru2_prom.PREMIUM IS NULL;
""")

### Máximo

In [21]:
#Crea tabla temporal del los posibles grupos de las combinaciones de las columas con el promedio
cursor.execute("""
-- Tabla temporal para guardar el promedio de la agrupación
CREATE TEMPORARY TABLE tabla_agrupa_max AS
SELECT
    PROD_YEAR,
    SEATS_NUM,
    TYPE_VEHICLE,
    rango_asegurado,
    MAX(PREMIUM) AS imput_PREMIUM_max
FROM
    Seguros_de_autos
WHERE
    PREMIUM IS NOT NULL
    
--Agrupa las filas de la tabla según las columnas especificadas   
--Es decir, va a calcular el promedio PREMIUM dentro de cada grupo
GROUP BY           
    PROD_YEAR, SEATS_NUM, TYPE_VEHICLE, rango_asegurado;
""")
con.commit()


In [22]:
#Crear una tabla temporal copia de seguros de autos para guardar la tabla con los datos imputados con con la agrupación y estadística especificada
cursor.execute("""
CREATE TEMPORARY TABLE tempora_gru2_max AS
SELECT *
FROM Seguros_de_autos;
""")

In [23]:
#modificar los nulos con el maximo
cursor.execute("""
UPDATE tempora_gru2_max
SET PREMIUM = (
    SELECT tabla_agrupa_max.imput_PREMIUM_max
    FROM tabla_agrupa_max
    WHERE
        tempora_gru2_max.PROD_YEAR = tabla_agrupa_max.PROD_YEAR AND
        tempora_gru2_max.SEATS_NUM = tabla_agrupa_max.SEATS_NUM AND
        tempora_gru2_max.rango_asegurado = tabla_agrupa_max.rango_asegurado AND
        tempora_gru2_max.TYPE_VEHICLE = tabla_agrupa_max.TYPE_VEHICLE
)
WHERE tempora_gru2_max.PREMIUM IS NULL;
""")


### Extraer las tablas de SQL a DataFrames en python

In [24]:
# SQL a python
Seguros_de_autos = pd.read_sql(
    '''
    SELECT *
    FROM Seguros_de_autos
    ''',
    con
)

df_grup1_prom = pd.read_sql(
    '''
    SELECT *
    FROM tempora_gru1_prom
    ''',
    con
)

df_grup1_max = pd.read_sql(
    '''
    SELECT *
    FROM tempora_gru1_max
    ''',
    con
)

df_grup2_prom = pd.read_sql(
    '''
    SELECT *
    FROM tempora_gru2_prom
    ''',
    con
)

df_grup2_max = pd.read_sql(
    '''
    SELECT *
    FROM tempora_gru2_max
    ''',
    con
)

#me traigo de SQL todas las tablas temporales a df a python

In [25]:
#Convierto las tablas en dataframes de pandas
Seguros_de_autos = pd.DataFrame(Seguros_de_autos)
df_grup1_prom = pd.DataFrame(df_grup1_prom)
df_grup1_max = pd.DataFrame(df_grup1_max)
df_grup2_prom = pd.DataFrame(df_grup2_prom)
df_grup2_max = pd.DataFrame(df_grup2_max)

# Métricas

## Grupo 1:

### Promedio

In [26]:
# Valores originales
y_true = np.array(df['PREMIUM']) #Se extrae la columna PREMIUM DEL df original

# Valores imputados
y_pred = np.array(df_grup1_prom['PREMIUM'])

# Enmascarar los valores no nulos https://numpy.org/doc/stable/reference/generated/numpy.isnan.html
mask = ~np.isnan(y_true) & ~np.isnan(y_pred)

# Filtrar los valores no nulos
y_true_filtrado = y_true[mask]
y_pred_filtrado = y_pred[mask]

# Calcular RSS
rss = np.sum((y_true_filtrado - y_pred_filtrado)**2)
print(f"RSS: {rss}")

# Calcular RMSE
rmse = np.sqrt(mean_squared_error(y_true_filtrado, y_pred_filtrado))
print(f"RMSE: {rmse}")

# Calcular R²
r2 = r2_score(y_true_filtrado, y_pred_filtrado)
print(f"R²: {r2}")


RSS: 3435222762469.8433
RMSE: 3431.8111606909
R²: 0.8557281264858414


### Máximo

In [31]:

y_pred = np.array(df_grup1_max['PREMIUM'])

mask = ~np.isnan(y_true) & ~np.isnan(y_pred)
y_true_filtrado = y_true[mask]
y_pred_filtrado = y_pred[mask]

rss = np.sum((y_true_filtrado - y_pred_filtrado)**2)
print(f"RSS: {rss}")

rmse = np.sqrt(mean_squared_error(y_true_filtrado, y_pred_filtrado))
print(f"RMSE: {rmse}")

r2 = r2_score(y_true_filtrado, y_pred_filtrado)
print(f"R²: {r2}")


RSS: 98466991228570.58
RMSE: 18373.473349252374
R²: -3.1353991534552783


## Grupo 2:

## Promedio

In [28]:
y_pred = np.array(df_grup2_prom['PREMIUM'])

mask = ~np.isnan(y_true) & ~np.isnan(y_pred)
y_true_filtrado = y_true[mask]
y_pred_filtrado = y_pred[mask]

rss = np.sum((y_true_filtrado - y_pred_filtrado)**2)
print(f"RSS: {rss}")

rmse = np.sqrt(mean_squared_error(y_true_filtrado, y_pred_filtrado))
print(f"RMSE: {rmse}")

r2 = r2_score(y_true_filtrado, y_pred_filtrado)
print(f"R²: {r2}")


RSS: 3705084352233.7363
RMSE: 3566.1510734461062
R²: 0.8442776314738616


In [ ]:
# Máximo

In [29]:
y_pred = np.array(df_grup2_max['PREMIUM'])

mask = ~np.isnan(y_true) & ~np.isnan(y_pred)
y_true_filtrado = y_true[mask]
y_pred_filtrado = y_pred[mask]

rss = np.sum((y_true_filtrado - y_pred_filtrado)**2)
print(f"RSS: {rss}")

rmse = np.sqrt(mean_squared_error(y_true_filtrado, y_pred_filtrado))
print(f"RMSE: {rmse}")

r2 = r2_score(y_true_filtrado, y_pred_filtrado)
print(f"R²: {r2}")


RSS: 50831680906657.234
RMSE: 13208.938648275243
R²: -1.13642362613886
